In [1]:
import pandas as pd
import spacy
import umap
import numpy as np 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
import matplotlib.pyplot as plt 
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

from ml_editor.data_processing import format_raw_df, get_split_by_author, add_text_features_to_df, get_vectorized_series, train_vectorizer

data_path=Path("D:\Project 1\data\writers.csv")
df=pd.read_csv(data_path)
df=format_raw_df(df.copy())
df=add_text_features_to_df(df.loc[df["is_question"]].copy())

train_author, test_author=get_split_by_author(df[df["is_question"]])


KeyError: 'is_question'

In [2]:
nlp=spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner", "textcat"])
train_author["vectors"]=train_author["full_text"].apply(lambda x: nlp(x).vector)
vectorized_features=np.vstack(train_author["vectors"])


OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
umap_embedder=umap.UMAP()
umap_features=umap_embedder.fit_transform(vectorized_features)

In [ ]:
n_clusters=3
cmap=plt.get_cmap("Set2")
fig=plt.figure(figsize=(16,10))

# Fit clustering algorithm to our vectorized features
clus=KMeans(n_clusters=n_clusters, random_state=10)
clusters=clus.fit_predict(vectorized_features)

# plot the dimensionality reduced features on a 2D plane
plt.scatter(umap_features[:, 0], umap_features[:, 1], c=[cmap(x/n_clusters)], s=40, alpha=4)
plt.title("UMAP projections of questions, colored by clusters", fontsize=14)

In [ ]:
clusters=[2, 3, 4, 5, 6]
to_clus=vectorized_features

for cluster in clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2)=plt.subplots(1, 2)

    # The first subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all lie within [-0.1, 1] 
    ax1.set_xlim([-0.3, .4])
    ax1.set_ylim([0, len(to_clus) + (n_clusters + 1) * 10])

    clusterer=KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels=clusterer.fit_predict(to_clus)



    silhouette_avg = silhouette_score(to_clus, cluster_labels, metric='cosine')
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(to_clus, cluster_labels, metric='cosine')

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cmap(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.3, -0.2, -0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cmap(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(umap_features[:, 0], umap_features[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centerss = clusterer.cluster_centers_
    print(centerss.shape)
    centers = umap_embedder.transform(centerss)
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')